In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210327.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,24/03/2021 10:43:33,RoboCop (1987),Cross of Iron (1977),River's Edge (1986),The Thing (1982),BASKETBALL,ACADEMY AWARD HOMEWORK,[Default to Nicolas Cage],I AM TERRIBLE AT MY JOB.,PRACTICAL EFFECTS
1,24/03/2021 10:51:15,RoboCop (1987),Cross of Iron (1977),The Thing (1982),River's Edge (1986),[Default to Nicolas Cage],I AM TERRIBLE AT MY JOB.,BASKETBALL,PRACTICAL EFFECTS,ACADEMY AWARD HOMEWORK
2,24/03/2021 10:57:20,RoboCop (1987),The Thing (1982),River's Edge (1986),Cross of Iron (1977),PRACTICAL EFFECTS,BASKETBALL,ACADEMY AWARD HOMEWORK,I AM TERRIBLE AT MY JOB.,[Default to Nicolas Cage]
3,24/03/2021 11:14:56,Cross of Iron (1977),River's Edge (1986),RoboCop (1987),The Thing (1982),I AM TERRIBLE AT MY JOB.,PRACTICAL EFFECTS,ACADEMY AWARD HOMEWORK,[Default to Nicolas Cage],BASKETBALL
4,24/03/2021 11:23:31,RoboCop (1987),The Thing (1982),Cross of Iron (1977),River's Edge (1986),BASKETBALL,PRACTICAL EFFECTS,I AM TERRIBLE AT MY JOB.,ACADEMY AWARD HOMEWORK,[Default to Nicolas Cage]
5,25/03/2021 08:21:13,RoboCop (1987),The Thing (1982),River's Edge (1986),Cross of Iron (1977),[Default to Nicolas Cage],PRACTICAL EFFECTS,I AM TERRIBLE AT MY JOB.,ACADEMY AWARD HOMEWORK,BASKETBALL
6,25/03/2021 08:27:29,River's Edge (1986),RoboCop (1987),Cross of Iron (1977),The Thing (1982),I AM TERRIBLE AT MY JOB.,PRACTICAL EFFECTS,BASKETBALL,ACADEMY AWARD HOMEWORK,[Default to Nicolas Cage]
7,25/03/2021 08:28:13,Cross of Iron (1977),River's Edge (1986),NaN,NaN,ACADEMY AWARD HOMEWORK,I AM TERRIBLE AT MY JOB.,PRACTICAL EFFECTS,BASKETBALL,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['RoboCop (1987)', 'Cross of Iron (1977)', "River's Edge (1986)",
       'The Thing (1982)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'RoboCop (1987)': <Candidate('RoboCop (1987)')>,
 'Cross of Iron (1977)': <Candidate('Cross of Iron (1977)')>,
 "River's Edge (1986)": <Candidate('River's Edge (1986)')>,
 'The Thing (1982)': <Candidate('The Thing (1982)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(RoboCop (1987), Cross of Iron (1977), River's Edge (1986), The Thing (1982))>,
 <Ballot(RoboCop (1987), Cross of Iron (1977), The Thing (1982), River's Edge (1986))>,
 <Ballot(RoboCop (1987), The Thing (1982), River's Edge (1986), Cross of Iron (1977))>,
 <Ballot(Cross of Iron (1977), River's Edge (1986), RoboCop (1987), The Thing (1982))>,
 <Ballot(RoboCop (1987), The Thing (1982), Cross of Iron (1977), River's Edge (1986))>,
 <Ballot(RoboCop (1987), The Thing (1982), River's Edge (1986), Cross of Iron (1977))>,
 <Ballot(River's Edge (1986), RoboCop (1987), Cross of Iron (1977), The Thing (1982))>,
 <Ballot(Cross of Iron (1977), River's Edge (1986))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate               Votes  Status
--------------------  -------  --------
RoboCop (1987)              5  Elected
Cross of Iron (1977)        2  Rejected
River's Edge (1986)         1  Rejected
The Thing (1982)            0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                    Votes  Status
-------------------------  -------  --------
I AM TERRIBLE AT MY JOB.         2  Hopeful
BASKETBALL                       2  Hopeful
[Default to Nicolas Cage]        2  Hopeful
PRACTICAL EFFECTS                1  Rejected
ACADEMY AWARD HOMEWORK           1  Rejected

ROUND 2
Candidate                    Votes  Status
-------------------------  -------  --------
I AM TERRIBLE AT MY JOB.         3  Hopeful
BASKETBALL                       3  Hopeful
[Default to Nicolas Cage]        2  Rejected
PRACTICAL EFFECTS                0  Rejected
ACADEMY AWARD HOMEWORK           0  Rejected

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
I AM TERRIBLE AT MY JOB.         5  Elected
BASKETBALL                       3  Rejected
[Default to Nicolas Cage]        0  Rejected
PRACTICAL EFFECTS                0  Rejected
ACADEMY AWARD HOMEWORK           0  Rejected

